In [ ]:
jittering = transforms.ColorJitter(brightness = (0.1,0.4), contrast = (0.1,0.4), saturation = (0.1,0.4))


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:

transform_train = transforms.Compose([transforms.ToTensor(),
  jittering
        ])





transform  = transforms.Compose([ transforms.ToTensor(),

        ])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/Statistical_Deep_Image",
                                           transform = transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

In [ ]:
net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [ ]:
for epoch in range(10):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
      



Epoch: 1,Batch : 750, Loss:0.7920472210049629
Epoch: 1,Batch : 1500, Loss:0.6843177570104599
Epoch: 2,Batch : 750, Loss:0.5877342796325684
Epoch: 2,Batch : 1500, Loss:0.5034955669939518
Epoch: 3,Batch : 750, Loss:0.4098104162812233
Epoch: 3,Batch : 1500, Loss:0.37009351898729803
Epoch: 4,Batch : 750, Loss:0.30713091406971216
Epoch: 4,Batch : 1500, Loss:0.29542321026325224
Epoch: 5,Batch : 750, Loss:0.24367058895528315
Epoch: 5,Batch : 1500, Loss:0.24070696695894003
Epoch: 6,Batch : 750, Loss:0.19173629730939865
Epoch: 6,Batch : 1500, Loss:0.19567097444459797
Epoch: 7,Batch : 750, Loss:0.15012819900736213
Epoch: 7,Batch : 1500, Loss:0.15796856589242816
Epoch: 8,Batch : 750, Loss:0.11138143332116306


pretrained 넣기


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')



import random
import math
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)

In [ ]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [ ]:
jittering = transforms.ColorJitter(brightness = 0.3, contrast = 0.3, saturation = 0.4)


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:

transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),

        ])





transform  = transforms.Compose([ transforms.ToTensor(),
                                                                       transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
        ])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/Statistical_Deep_Image",
                                           transform = transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)


test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

모델 넣기 

In [ ]:
class BasicBlock(nn.Module):
   def __init__(self, in_channels, out_channels, kernel_size=3):
       super(BasicBlock, self).__init__()


       # ❶ 합성곱층 정의
       self.c1 = nn.Conv2d(in_channels, out_channels, 
                           kernel_size=kernel_size, padding=1)
       self.c2 = nn.Conv2d(out_channels, out_channels, 
                           kernel_size=kernel_size, padding=1)

       self.downsample = nn.Conv2d(in_channels, out_channels, 
                                   kernel_size=1)
       
       # ❷ 배치 정규화층 정의
       self.bn1 = nn.BatchNorm2d(num_features=out_channels)
       self.bn2 = nn.BatchNorm2d(num_features=out_channels)

       self.relu = nn.ReLU()
   def forward(self, x):
       # ❸스킵 커넥션을 위해 초기 입력을 저장
       x_ = x

       x = self.c1(x)
       x = self.bn1(x)
       x = self.relu(x)
       x = self.c2(x)
       x = self.bn2(x)

       # ➍합성곱의 결과와 입력의 채널 수를 맞춤
       x_ = self.downsample(x_)

       # ➎합성곱층의 결과와 저장해놨던 입력값을 더해줌
       x += x_
       x = self.relu(x)

       return x


class ResNet(nn.Module):
   def __init__(self, num_classes=10):
       super(ResNet, self).__init__()


       # ❶ 기본 블록
       self.b1 = BasicBlock(in_channels=3, out_channels=64)
       self.b2 = BasicBlock(in_channels=64, out_channels=128)
       self.b3 = BasicBlock(in_channels=128, out_channels=256)


       # ❷ 풀링을 최댓값이 아닌 평균값으로
       self.pool = nn.AvgPool2d(kernel_size=2, stride=2) 

       # ❸ 분류기
       self.fc1 = nn.Linear(in_features=4096, out_features=2048)
       self.fc2 = nn.Linear(in_features=2048, out_features=512)
       self.fc3 = nn.Linear(in_features=512, out_features=num_classes)

       self.relu = nn.ReLU()
   def forward(self, x):
       # ❶ 기본 블록과 풀링층을 통과
       x = self.b1(x)
       x = self.pool(x)
       x = self.b2(x)
       x = self.pool(x)
       x = self.b3(x)
       x = self.pool(x)


       # ❷ 분류기의 입력으로 사용하기 위해 flatten
       x = torch.flatten(x, start_dim=1)

       # ❸ 분류기로 예측값 출력
       x = self.fc1(x)
       x = self.relu(x)
       x = self.fc2(x)
       x = self.relu(x)
       x = self.fc3(x)

       return x

In [ ]:
net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.000001)

In [ ]:
for epoch in range(10):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0



Epoch: 1,Batch : 750, Loss:0.5604290823638439
Epoch: 1,Batch : 1500, Loss:0.3965580953657627
Epoch: 2,Batch : 750, Loss:0.29546111458539964
Epoch: 2,Batch : 1500, Loss:0.258781103014946
Epoch: 3,Batch : 750, Loss:0.19565488464385272
Epoch: 3,Batch : 1500, Loss:0.1893154518455267
Epoch: 4,Batch : 750, Loss:0.12729302999190986
Epoch: 4,Batch : 1500, Loss:0.13379417736269533
Epoch: 5,Batch : 750, Loss:0.08211038008052855
Epoch: 5,Batch : 1500, Loss:0.09357821545749903
Epoch: 6,Batch : 750, Loss:0.049738563649589194
Epoch: 6,Batch : 1500, Loss:0.061254371469840405
Epoch: 7,Batch : 750, Loss:0.039112058409664315
Epoch: 7,Batch : 1500, Loss:0.04363377985497936
Epoch: 8,Batch : 750, Loss:0.0303137074377737
Epoch: 8,Batch : 1500, Loss:0.03550255238701357
Epoch: 9,Batch : 750, Loss:0.026535421120759566
Epoch: 9,Batch : 1500, Loss:0.03083486866115709
Epoch: 10,Batch : 750, Loss:0.022291735005768715
Epoch: 10,Batch : 1500, Loss:0.028727351570341852


pretrained 넣기


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaild Accuracy  : {}".format(100* correct/total))

trainset Accuracy  : 81.9


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

testset Accuracy  : 47.05


In [ ]:
jittering = transforms.ColorJitter(brightness = 0.3, contrast = 0.3, saturation = 0.4)


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:

transform_train = transforms.Compose([transforms.ToTensor(),
 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
        ])





transform  = transforms.Compose([ transforms.ToTensor(),
                                  transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
        ])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/Statistical_Deep_Image",
                                           transform = transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

In [ ]:
net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [ ]:
for epoch in range(10):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
      



Epoch: 1,Batch : 750, Loss:0.5742686148583889
Epoch: 1,Batch : 1500, Loss:0.4106494539231062
Epoch: 2,Batch : 750, Loss:0.31436835338175295
Epoch: 2,Batch : 1500, Loss:0.2706271687820554
Epoch: 3,Batch : 750, Loss:0.20704827290028333
Epoch: 3,Batch : 1500, Loss:0.19835794933885337
Epoch: 4,Batch : 750, Loss:0.1424439019560814
Epoch: 4,Batch : 1500, Loss:0.14086420067958533
Epoch: 5,Batch : 750, Loss:0.08884132007323206
Epoch: 5,Batch : 1500, Loss:0.09541876823827625
Epoch: 6,Batch : 750, Loss:0.05447366884723306
Epoch: 6,Batch : 1500, Loss:0.0655631344676949
Epoch: 7,Batch : 750, Loss:0.03749638847797178
Epoch: 7,Batch : 1500, Loss:0.04783218496257905
Epoch: 8,Batch : 750, Loss:0.028435727234435035
Epoch: 8,Batch : 1500, Loss:0.036793885451334066
Epoch: 9,Batch : 750, Loss:0.023062976538902148
Epoch: 9,Batch : 1500, Loss:0.03179990821040701
Epoch: 10,Batch : 750, Loss:0.023008025035989702
Epoch: 10,Batch : 1500, Loss:0.02794325759138883


pretrained 넣기


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

vaildset Accuracy  : 81.42


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

testset Accuracy  : 43.1


In [ ]:
jittering = transforms.ColorJitter(brightness = (0.1,0.4), contrast = (0.1,0.4), saturation = (0.1,0.4))


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:

transform_train = transforms.Compose([transforms.ToTensor(),
  jittering
        ])





transform  = transforms.Compose([ transforms.ToTensor(),

        ])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/Statistical_Deep_Image",
                                           transform = transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

In [ ]:
net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [ ]:
for epoch in range(10):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
      



Epoch: 1,Batch : 750, Loss:0.7920472210049629
Epoch: 1,Batch : 1500, Loss:0.6843177570104599
Epoch: 2,Batch : 750, Loss:0.5877342796325684
Epoch: 2,Batch : 1500, Loss:0.5034955669939518
Epoch: 3,Batch : 750, Loss:0.4098104162812233
Epoch: 3,Batch : 1500, Loss:0.37009351898729803
Epoch: 4,Batch : 750, Loss:0.30713091406971216
Epoch: 4,Batch : 1500, Loss:0.29542321026325224
Epoch: 5,Batch : 750, Loss:0.24367058895528315
Epoch: 5,Batch : 1500, Loss:0.24070696695894003
Epoch: 6,Batch : 750, Loss:0.19173629730939865
Epoch: 6,Batch : 1500, Loss:0.19567097444459797
Epoch: 7,Batch : 750, Loss:0.15012819900736213
Epoch: 7,Batch : 1500, Loss:0.15796856589242816
Epoch: 8,Batch : 750, Loss:0.11138143332116306
Epoch: 8,Batch : 1500, Loss:0.12226440766081213
Epoch: 9,Batch : 750, Loss:0.08254361521266401
Epoch: 9,Batch : 1500, Loss:0.09834219278208911
Epoch: 10,Batch : 750, Loss:0.06141183136124164
Epoch: 10,Batch : 1500, Loss:0.07319813255453482


pretrained 넣기


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

vaildset Accuracy  : 74.14


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

testset Accuracy  : 42.05


In [ ]:
jittering = transforms.ColorJitter(brightness = (0.1,0.4), contrast = (0.1,0.4), saturation = (0.1,0.4))


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:

transform_train = transforms.Compose([transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.5)
        ])





transform  = transforms.Compose([ transforms.ToTensor(),

        ])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=True,
                                        transform=transform_train)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=True,
                                        transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/Statistical_Deep_Image",
                                           transform = transform)

In [ ]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

In [ ]:
net = ResNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

total=0 
correct = 0

In [ ]:
for epoch in range(10):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
      



Epoch: 1,Batch : 750, Loss:0.5877426655590534
Epoch: 1,Batch : 1500, Loss:0.4274163376092911
Epoch: 2,Batch : 750, Loss:0.3346704117059708
Epoch: 2,Batch : 1500, Loss:0.283121898651123
Epoch: 3,Batch : 750, Loss:0.23469326613098382
Epoch: 3,Batch : 1500, Loss:0.2157737657725811
Epoch: 4,Batch : 750, Loss:0.17674002100899816
Epoch: 4,Batch : 1500, Loss:0.175645044259727
Epoch: 5,Batch : 750, Loss:0.14346168457344174
Epoch: 5,Batch : 1500, Loss:0.14264629148319363
Epoch: 6,Batch : 750, Loss:0.11532783246412873
Epoch: 6,Batch : 1500, Loss:0.1194775180220604
Epoch: 7,Batch : 750, Loss:0.09187927455734461
Epoch: 7,Batch : 1500, Loss:0.09957177088037134
Epoch: 8,Batch : 750, Loss:0.07512747709918767
Epoch: 8,Batch : 1500, Loss:0.0822572408085689
Epoch: 9,Batch : 750, Loss:0.06276898121333216
Epoch: 9,Batch : 1500, Loss:0.07087478585727512
Epoch: 10,Batch : 750, Loss:0.05445669889682904
Epoch: 10,Batch : 1500, Loss:0.05837930965819396


pretrained 넣기


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

vaildset Accuracy  : 84.61


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

testset Accuracy  : 48.45
